In [ ]:
for text in llm("AI is going to", stream=True):
    print(text, end="", flush=True)


# results on the cpu using ctransformers
### - 27.4 seconds
### output:
<p>
have an enormous impact on the future of the industry.
This year, AI was all over the show floor and there are a number of companies that are using AI in their products. Here's a quick look at some of them:
Fulcrum is an automated, artificial intelligence-powered software solution that helps manufacturers improve productivity and efficiency by automating processes, removing errors, and freeing up employees to focus on more complex tasks.
Kapil Khera, co-founder and CEO of Fulcrum, told Inman: "I don't know a single company that doesn't have AI in their product. I think it's the most important thing to come out of 2018."
The company is currently working on machine learning tools and workflow applications with machine intelligence technology so you can really begin building these powerful AI-powered solutions. The full AI-focused ecosystem, with automation in product design and sophisticated analytics, will be available to all customers by the end of 2018.
Kapil Khera, co-founder and CEO of Fulcrum
Surely
</p>

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "TheBloke/TinyLlama-1.1B-Chat-v1.0-GPTQ"
# To use a different branch, change revision
# For example: revision="gptq-4bit-32g-actorder_True"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)


In [ ]:

def get_response_tiny_llama(text):
    path='/teamspace/studios/this_studio/TinyLlama-1.1B-Chat-v1.0-GPTQ'
    model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
    prompt = text
    system_message = "You are a story writing assistant"
    prompt_template=f'''<|system|>
    {system_message}</s>
    <|user|>
    {prompt}</s>
    <|assistant|>
    '''

    print("\n\n*** Generate:")

    input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
    output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
    return(tokenizer.decode(output[0]))

In [ ]:
print(get_response_tiny_llama("what is qlora?"))

# result on gpu 2.8 seconds

# handling the embedding model

In [ ]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')
embeddings = model.encode(sentences)
print(embeddings)


In [3]:
def convert_from_pdf_to_text(file_path):
    import PyPDF2

    read_pdf = PyPDF2.PdfReader(file_path)
    pages = read_pdf.pages
    res=''
    for page in pages:
        res+=page.extract_text()
    return res
pages=convert_from_pdf_to_text('/Users/matansharon/python/opensource_rag/pdf_files/A semantic loss for ontology classification.pdf')

import os
pdf_list=os.listdir("/Users/matansharon/python/opensource_rag/pdf_files")
text_list=os.listdir("/Users/matansharon/python/opensource_rag/text_files")

for pdf in pdf_list:
    temp=pdf.replace(".pdf",".txt")
    if temp not in text_list:
        text=convert_from_pdf_to_text("pdf_files/"+pdf)
        with open("text_files/"+temp, 'w') as f:
            f.write(text)

In [ ]:
def convert_from_pdf_to_text(file_path):
    import PyPDF2

    read_pdf = PyPDF2.PdfReader(file_path)
    pages = read_pdf.pages
    res=''
    for page in pages:
        res+=page.extract_text()
    return res
pages=convert_from_pdf_to_text('/Users/matansharon/python/opensource_rag/pdf_files/A semantic loss for ontology classification.pdf')


In [ ]:
import chromadb
client=chromadb.Client()
import os
import chromadb.utils.embedding_functions as embedding_functions
huggingface_ef = embedding_functions.HuggingFaceEmbeddingFunction(
    api_key=os.environ.get("HuggingFace_API_KEY"),
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

collection=client.get_or_create_collection('test',embedding_function=huggingface_ef)


In [1]:
import chromadb
persist_directory = "chroma_storage"
client = chromadb.PersistentClient(path=persist_directory)
collection=client.get_collection("file1_collection")
print(collection.get().keys())



dict_keys(['ids', 'embeddings', 'metadatas', 'documents', 'uris', 'data'])


In [10]:
all_file_names=set()
for line in collection.get()['metadatas']:
    all_file_names.add(line['filename'])
all_file_names

{'qlora.txt'}

In [37]:
for doc in collection.get()['documents']:
    print(len(doc))

60
26
87
87
50
49
48
30
46
44
52
105
98
11
105
54
55
34
43
39
40
39
41
51
97
107
78
31
35
27
25
18
10
19
19
79
60
55
59
66
64
66
64
38
64
59
54
73
62
58
93
93
90
99
92
66
84
92
26
94
95
68
101
101
98
91
98
82
100
59
48
49
96
100
101
101
106
59
25
30
71
67
53
51
57
88
46
22
86
100
99
56
104
97
79
96
97
104
97
92
98
91
77
93
92
95
60
92
93
85
90
98
18
91
84
14
47
45
21
22
35
33
28
25
27
44
38
36
26
75
50
54
54
51
54
21
82
53
49
54
53
56
53
52
92
6
94
63
99
100
99
99
77
94
96
95
101
98
84
93
100
92
91
94
97
89
90
100
93
94
96
90
54
91
95
99
96
94
96
97
27
94
98
99
94
98
99
98
67
92
95
93
90
95
52
97
94
96
94
96
86
94
106
99
100
95
98
105
103
67
82
38
33
88
47
45
46
46
54
45
46
46
46
45
93
107
46
55
44
47
52
46
44
43
44
91
95
98
91
93
90
95
82
94
89
93
93
61
92
93
96
90
43
101
98
90
96
96
111
90
96
71
22
97
101
101
97
103
104
91
99
108
50
38
19
42
32
28
34
33
62
38
30
31
31
22
94
94
46
47
99
14
94
98
98
90
100
82
100
103
102
100
109
97
50
93
31
41
90
55
75
14
57
112
62
5
92
45
94
51
87
11
